In [1]:
import pandas as pd

df = pd.read_csv('data/skylab_instagram_datathon_dataset.csv', delimiter=';', na_values=['NA', 'N/A', '-', 'NaN'])



In [2]:
df.head()

,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
0,Weekly,2019-12-07,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,22066189.0,97.0,13.0,16239.0,6112205.0
1,Weekly,2021-01-30,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,24289963.0,54.0,2.0,13358.0,3189144.0
2,Weekly,2016-05-14,Luxury & Premium & Mainstream,Soft Luxury,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,7137442.0,42.0,8.0,15894.0,2152955.0
3,Weekly,2019-04-20,Luxury & Premium & Mainstream,Luxury & Premium & Mainstream,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,18901125.0,73.0,9.0,26762.0,5224916.0
4,Weekly,2023-03-25,Luxury & Premium & Mainstream,US Softlines Analyst Interest List,Versace,Capri Holdings,United States of America,Capri Holdings,New York Stock Exchange,Metric Value,30251374.0,54.0,10.0,25681.0,5760987.0


In [3]:
df.tail(5)

,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
704308,Weekly,2017-03-18,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,NaN,50.0,8.0,1271.0,14818.0
704309,Weekly,2018-10-27,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,163873.0,125.0,2.0,1764.0,34188.0
704310,Weekly,2023-08-05,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,319485.0,118.0,35.0,1697.0,6328.0
704311,Weekly,2019-11-02,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,228699.0,129.0,2.0,1972.0,29174.0
704312,Weekly,2023-02-25,Outdoor Gear,Outdoor Gear,Klean Kanteen,Klean Kanteen Inc,NaN,Klean Kanteen Inc,NaN,Metric Value,310895.0,128.0,33.0,2606.0,10960.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704313 entries, 0 to 704312
Data columns (total 15 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   period                                  704313 non-null  object 
 1   period_end_date                         704313 non-null  object 
 2   compset_group                           704313 non-null  object 
 3   compset                                 704313 non-null  object 
 4   business_entity_doing_business_as_name  704313 non-null  object 
 5   legal_entity_name                       676558 non-null  object 
 6   domicile_country_name                   458589 non-null  object 
 7   ultimate_parent_legal_entity_name       676558 non-null  object 
 8   primary_exchange_name                   458589 non-null  object 
 9   calculation_type                        704313 non-null  object 
 10  followers                               5792

In [5]:

#list of columns
columns = list(df.columns)

#finding unique values in each field.
# period : no point only weekly
# period_end_date : unique date values
# compset_group
# compset
# business_entity_doing_business_as_name
# legal_entity_name
# domicile_country_name
# ultimate_parent_legal_entity_name
# primary_e|xchange_name
# calculation_type : metric values
# followers : unique number
# pictures : unique number
# videos : unique number
# comments : unique number
# like : unique number

for column in columns:
    print('--------------------Column name: {}'. format(column))
    unique_values = df[column].unique()
    print('Number of unique elements: {} '.format(len(unique_values)))
    

--------------------Column name: period
Number of unique elements: 1 
--------------------Column name: period_end_date
Number of unique elements: 455 
--------------------Column name: compset_group
Number of unique elements: 20 
--------------------Column name: compset
Number of unique elements: 54 
--------------------Column name: business_entity_doing_business_as_name
Number of unique elements: 706 
--------------------Column name: legal_entity_name
Number of unique elements: 424 
--------------------Column name: domicile_country_name
Number of unique elements: 27 
--------------------Column name: ultimate_parent_legal_entity_name
Number of unique elements: 402 
--------------------Column name: primary_exchange_name
Number of unique elements: 31 
--------------------Column name: calculation_type
Number of unique elements: 1 
--------------------Column name: followers
Number of unique elements: 243232 
--------------------Column name: pictures
Number of unique elements: 8191 
--------

In [6]:
#ignoring the extra stuff temporarily 
for column in columns[2:9]:
    print('--------------------Column name: {}'. format(column))
    unique_values = df[column].unique()
    print('Number of unique elements: {} '.format(len(unique_values)))

--------------------Column name: compset_group
Number of unique elements: 20 
--------------------Column name: compset
Number of unique elements: 54 
--------------------Column name: business_entity_doing_business_as_name
Number of unique elements: 706 
--------------------Column name: legal_entity_name
Number of unique elements: 424 
--------------------Column name: domicile_country_name
Number of unique elements: 27 
--------------------Column name: ultimate_parent_legal_entity_name
Number of unique elements: 402 
--------------------Column name: primary_exchange_name
Number of unique elements: 31 


In [7]:
unique_compset_group = df["compset_group"].unique()
print(unique_compset_group)
# print(df_nan[df_nan['name'].str.contains('li')])
df_compset_group = (df[df['compset_group'] == 'Restaurants'])
df_compset_group.head()
# print(len(df_compset_group))

['Luxury & Premium & Mainstream' 'Restaurants' 'Beverages'
 'Apparel Retail' 'Food Products' 'Sporting Goods'
 'Sportswear & Athleisure' 'Beauty & Boutique' 'Building Products'
 'Petcare' 'Toys & Collectibles' 'Mattress' 'Fitness & Exercise'
 'Food Retail' 'Discount Retailers ' 'Outdoor Gear'
 'Dermatology and Orthodontics' 'Entertainment' 'Home Appliances'
 'Study (All Brands)']


,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
1820,Weekly,2020-10-31,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,563920.0,75.0,24.0,2635.0,49433.0
1821,Weekly,2019-01-19,Restaurants,Restaurants,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,367410.0,33.0,18.0,5563.0,40775.0
1822,Weekly,2016-11-19,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,216904.0,66.0,14.0,984.0,61223.0
1823,Weekly,2023-07-22,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,798579.0,47.0,44.0,3230.0,64806.0
1824,Weekly,2019-03-23,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,382849.0,30.0,21.0,1566.0,28498.0


In [8]:
df_compset_group = df_compset_group.reset_index()
df_compset_group.head()



,index,period,period_end_date,compset_group,compset,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,calculation_type,followers,pictures,videos,comments,likes
0,1820,Weekly,2020-10-31,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,563920.0,75.0,24.0,2635.0,49433.0
1,1821,Weekly,2019-01-19,Restaurants,Restaurants,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,367410.0,33.0,18.0,5563.0,40775.0
2,1822,Weekly,2016-11-19,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,216904.0,66.0,14.0,984.0,61223.0
3,1823,Weekly,2023-07-22,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,798579.0,47.0,44.0,3230.0,64806.0
4,1824,Weekly,2019-03-23,Restaurants,Coffee,Tim Horton's,Restaurant Brands International,Canada,Restaurant Brands International,Toronto Stock Exchange,Metric Value,382849.0,30.0,21.0,1566.0,28498.0


In [ ]:
#New strat: find brands with highest likes per post ---> see historical